In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
import string

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
nltk.download('stopwords')

In [ ]:
df = pd.read_csv("train-data.tsv", sep="\t")
df.columns = ['label', 'text']
df

In [ ]:
df['text'] = df['text'].apply(lambda x: x.replace('\r\n', ' '))
df
df.text.iloc[1]

In [ ]:
stemmer = PorterStemmer() # reduce words to their root form (e.g., "running" to "run").
corpus = [] # an empty list to store the processed text data

# Creates a set of common English stopwords (e.g., "the", "and", "is") that will be removed from the text.
stopwords_set = set(stopwords.words('english'))

for i in range(len(df)):
    text = df['text'].iloc[i].lower() # lower case all the text

    # remove all punctuations and splits it into individual words
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.split()

    # Applies stemming to each word in the text and removes stopwords.
    text = [stemmer.stem(word) for word in text if word not in stopwords_set]

    text = ' '.join(text)
    corpus.append(text)

In [ ]:
# # Initialize the CountVectorizer
# vectorizer = CountVectorizer()

# # Transform the corpus into a document-term matrix
# X = vectorizer.fit_transform(corpus).toarray()
# Y = df.label #FIXED

# FIXED
vectorizer = TfidfVectorizer(ngram_range=(1, 2))  # Using bi-grams
X = vectorizer.fit_transform(corpus)
Y = df['label'].apply(lambda x: 1 if x == 'spam' else 0)

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
# clf = RandomForestClassifier(n_jobs=-1)
clf = LogisticRegression(solver='liblinear')
clf.fit(X_train, Y_train)

In [ ]:
clf.score(X_test, Y_test)

In [ ]:
email_to_classify = df.text.values[10]
email_text = email_to_classify.lower().translate(str.maketrans('','',string.punctuation)).split()
email_text = [stemmer.stem(word) for word in text if word not in stopwords_set] # Stem and Remove Stopwords
email_text = ' '.join(email_text) # Combines the processed words back into a single string.

email_corpus = [email_text] # Puts the processed email text into a list.

X_email = vectorizer.transform(email_corpus)

In [ ]:
clf.predict(X_email)

df.label.iloc[10]

# the prediction is correct

In [ ]:
def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.split()
    text = [stemmer.stem(word) for word in text if word not in stopwords_set]
    return ' '.join(text)

You should create a function called predict_message that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the SMS Spam Collection dataset. The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    pred_text = pred_text.lower()
    pred_text = pred_text.translate(str.maketrans('', '', string.punctuation))
    pred_text = pred_text.split()
    pred_text = [stemmer.stem(word) for word in pred_text if word not in stopwords_set]
    pred_text = ' '.join(pred_text)

    X_pred = vectorizer.transform([pred_text])
    prediction_probability = clf.predict_proba(X_pred)[0][1]
    prediction_label = "spam" if prediction_probability > 0.5 else "ham"
    result = [prediction_probability, prediction_label]
    print(result)
    return result

# pred_text = "how are you doing today?"

# prediction = predict_message(pred_text)
# print(prediction)

test_messages = ["how are you doing today",
                  "sale today! to stop texts call 98912460324",
                  "i dont want to go. can we try it a different day? available sat",
                  "our new mobile video service is live. just install on your phone to start watching.",
                  "you have won £1000 cash! call to claim your prize.",
                  "i'll bring it tomorrow. don't forget the milk.",
                  "wow, is your arm alright. that happened to me one time too"
                ]

for msg in test_messages:
    print(msg)
    prediction = predict_message(msg)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
